In [5]:
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [6]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [7]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]


In [8]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,4,16)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
ohlc_df

,time,open,high,low,close
0,2025-04-15 23:00:00,1.12823,1.12848,1.12823,1.12848
1,2025-04-15 23:01:00,1.12847,1.12847,1.12824,1.12839
2,2025-04-15 23:02:00,1.12838,1.12862,1.12837,1.12862
3,2025-04-15 23:03:00,1.12862,1.12862,1.12837,1.12848
4,2025-04-15 23:04:00,1.12848,1.12848,1.12836,1.12839
...,...,...,...,...,...
62477,2025-06-16 09:26:00,1.15486,1.15496,1.15474,1.15474
62478,2025-06-16 09:27:00,1.15474,1.15474,1.15416,1.15427
62479,2025-06-16 09:28:00,1.15427,1.15439,1.15405,1.15417
62480,2025-06-16 09:29:00,1.15417,1.15429,1.15405,1.15429


In [9]:
# hour4 df
timeframe = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
hourly_df.tail(5)

,time,open,high,low,close
256,2025-06-13 16:00:00,1.15161,1.15696,1.15013,1.15545
257,2025-06-13 20:00:00,1.15545,1.15555,1.15336,1.15512
258,2025-06-16 00:00:00,1.15250,1.15480,1.15211,1.15348
259,2025-06-16 04:00:00,1.15343,1.15444,1.15237,1.15359
260,2025-06-16 08:00:00,1.15360,1.15865,1.15358,1.15780


In [10]:
# daily df
timeframe = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
daily_df

,time,open,high,low,close
0,2025-04-16,1.12781,1.14128,1.12781,1.13985
1,2025-04-17,1.13986,1.14093,1.13353,1.13634
2,2025-04-18,1.13616,1.13975,1.13586,1.13929
3,2025-04-21,1.13928,1.15733,1.13799,1.15129
4,2025-04-22,1.15139,1.15474,1.14175,1.14206
5,2025-04-23,1.14207,1.14399,1.13081,1.13145
6,2025-04-24,1.13114,1.13979,1.13095,1.13888
7,2025-04-25,1.13852,1.13914,1.13160,1.13595
8,2025-04-28,1.13721,1.14249,1.13294,1.14216
9,2025-04-29,1.14197,1.14218,1.13702,1.13828


In [11]:
# Calculate Pivot Point, R1, R2, R3, stopeR4, S1, S2, S3, stopeS4 levels with Fibonacci 
daily_df["pivot"] = ((daily_df["high"] + daily_df["low"] + daily_df["close"]) / 3).shift(1)
daily_df["prange"] = (daily_df["high"] - daily_df["low"]).shift(1)
daily_df["R1"] = daily_df["pivot"] + 0.382 * daily_df["prange"]
daily_df["R2"] = daily_df["pivot"] + 0.618 * daily_df["prange"]
daily_df["R3"] = daily_df["pivot"] + 1.000 * daily_df["prange"]

daily_df["S1"] = daily_df["pivot"] - 0.382 * daily_df["prange"]
daily_df["S2"] = daily_df["pivot"] - 0.618 * daily_df["prange"]
daily_df["S3"] = daily_df["pivot"] - 1.000 * daily_df["prange"]
daily_df[["time","S2","R2"]].tail(10)


,time,S2,R2
34,2025-06-03,1.134143,1.147937
35,2025-06-04,1.134047,1.145233
36,2025-06-05,1.135421,1.145012
37,2025-06-06,1.139197,1.150383
38,2025-06-09,1.135451,1.146056
39,2025-06-10,1.138270,1.144784
40,2025-06-11,1.136900,1.146133
41,2025-06-12,1.140628,1.152259
42,2025-06-13,1.147313,1.165853
43,2025-06-16,1.147388,1.162925


In [12]:
#setting the sma 1 and 14 periods
furst_sma_period = 1
second_sma_period = 14
#ohlc4
hourly_df['ohlc4'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4

# sma 1
hourly_df['sma_1'] = hourly_df['ohlc4'].rolling(furst_sma_period).mean()
hourly_df['prev_sma_1'] = hourly_df['sma_1'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 14
hourly_df['sma_14'] = hourly_df['ohlc4'].rolling(second_sma_period).mean()
hourly_df

,time,open,high,low,close,ohlc4,sma_1,prev_sma_1,sma_14
0,2025-04-16 00:00:00,1.12781,1.13217,1.12781,1.13169,1.129870,1.129870,NaN,NaN
1,2025-04-16 04:00:00,1.13169,1.13479,1.13164,1.13462,1.133185,1.133185,1.129870,NaN
2,2025-04-16 08:00:00,1.13462,1.13921,1.13384,1.13696,1.136158,1.136158,1.133185,NaN
3,2025-04-16 12:00:00,1.13697,1.13777,1.13408,1.13519,1.136003,1.136003,1.136158,NaN
4,2025-04-16 16:00:00,1.13516,1.13871,1.13498,1.13750,1.136588,1.136588,1.136003,NaN
...,...,...,...,...,...,...,...,...,...
256,2025-06-13 16:00:00,1.15161,1.15696,1.15013,1.15545,1.153538,1.153538,1.151747,1.152972
257,2025-06-13 20:00:00,1.15545,1.15555,1.15336,1.15512,1.154870,1.154870,1.153538,1.153664
258,2025-06-16 00:00:00,1.15250,1.15480,1.15211,1.15348,1.153223,1.153223,1.154870,1.154073
259,2025-06-16 04:00:00,1.15343,1.15444,1.15237,1.15359,1.153458,1.153458,1.153223,1.154397


In [13]:
#setting the 11 and 114 sma periods
thurd_sma_period = 1
fourth_sma_period = 14
#ohlc4
daily_df['ohlc4'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
# sma 11
daily_df['sma_11'] = daily_df['ohlc4'].rolling(thurd_sma_period).mean()
daily_df['prev_sma_11'] = daily_df['sma_11'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 114
daily_df['sma_114'] = daily_df['ohlc4'].rolling(fourth_sma_period).mean()
daily_df[["time","sma_11","sma_114","prev_sma_11"]]

,time,sma_11,sma_114,prev_sma_11
0,2025-04-16,1.134188,NaN,NaN
1,2025-04-17,1.137665,NaN,1.134188
2,2025-04-18,1.137765,NaN,1.137665
3,2025-04-21,1.146473,NaN,1.137765
4,2025-04-22,1.147485,NaN,1.146473
5,2025-04-23,1.137080,NaN,1.147485
6,2025-04-24,1.135190,NaN,1.137080
7,2025-04-25,1.136302,NaN,1.135190
8,2025-04-28,1.138700,NaN,1.136302
9,2025-04-29,1.139863,NaN,1.138700


In [14]:
#setting entring and closing time and one trade a day
# Set the start and end times for your desired trading hours
start_time = datetime.strptime('06:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('17:00:00', '%H:%M:%S').time()
end_trading_section = datetime.strptime('23:45:00', '%H:%M:%S').time()

# Filter the trade data for trades within the specified trading hours
#trades_within_hours = daily_df[(daily_df['time'].dt.time >= start_time) & (daily_df['time'].dt.time <= end_time)]
# open one trade a day
first_trade_per_day = daily_df.groupby(daily_df['time'].dt.time).first()
# Close all open positions at 11:45 PM
open_positions = daily_df[daily_df['time'].dt.time == end_trading_section]

# function to trading
daily_df["sell_stope_loss"] = (daily_df["R2"] - daily_df["S1"]) /2 + daily_df["R2"] 
daily_df["buy_stope_loss"] = (daily_df["S2"] - ((daily_df["R1"] - daily_df["S2"]) /2))
daily_df['sell_profit'] = daily_df['low'] <= daily_df["S1"]
daily_df['buy_profit'] = daily_df["high"] >= daily_df["R1"]
daily_df

,time,open,high,low,close,pivot,prange,R1,R2,R3,...,S2,S3,ohlc4,sma_11,prev_sma_11,sma_114,sell_stope_loss,buy_stope_loss,sell_profit,buy_profit
0,2025-04-16,1.12781,1.14128,1.12781,1.13985,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.134188,1.134188,NaN,NaN,NaN,NaN,False,False
1,2025-04-17,1.13986,1.14093,1.13353,1.13634,1.136313,0.01347,1.141459,1.144638,1.149783,...,1.127989,1.122843,1.137665,1.137665,1.134188,NaN,1.151373,1.121254,False,False
2,2025-04-18,1.13616,1.13975,1.13586,1.13929,1.136933,0.00740,1.139760,1.141507,1.144333,...,1.132360,1.129533,1.137765,1.137765,1.137665,NaN,1.145207,1.128660,False,False
3,2025-04-21,1.13928,1.15733,1.13799,1.15129,1.138300,0.00389,1.139786,1.140704,1.142190,...,1.135896,1.134410,1.146473,1.146473,1.137765,NaN,1.142649,1.133951,False,True
4,2025-04-22,1.15139,1.15474,1.14175,1.14206,1.148870,0.01934,1.156258,1.160822,1.168210,...,1.136918,1.129530,1.147485,1.147485,1.146473,NaN,1.170492,1.127248,False,False
5,2025-04-23,1.14207,1.14399,1.13081,1.13145,1.146183,0.01299,1.151146,1.154211,1.159173,...,1.138156,1.133193,1.137080,1.137080,1.147485,NaN,1.160706,1.131661,True,False
6,2025-04-24,1.13114,1.13979,1.13095,1.13888,1.135417,0.01318,1.140451,1.143562,1.148597,...,1.127271,1.122237,1.135190,1.135190,1.137080,NaN,1.150152,1.120681,False,False
7,2025-04-25,1.13852,1.13914,1.13160,1.13595,1.136540,0.00884,1.139917,1.142003,1.145380,...,1.131077,1.127700,1.136302,1.136302,1.135190,NaN,1.146423,1.126657,True,False
8,2025-04-28,1.13721,1.14249,1.13294,1.14216,1.135563,0.00754,1.138444,1.140223,1.143103,...,1.130904,1.128023,1.138700,1.138700,1.136302,NaN,1.143993,1.127134,False,True
9,2025-04-29,1.14197,1.14218,1.13702,1.13828,1.139197,0.00955,1.142845,1.145099,1.148747,...,1.133295,1.129647,1.139863,1.139863,1.138700,NaN,1.149874,1.128520,False,False


In [15]:
#ohlc_df = ohlc_df.sort_values('time')
#daily_df = daily_df.sort_values('time')
def entring_stoploss_profit(row, hourly_df) :
    if row["prev_sma_11"] > row['sma_114'] and row["prev_sma_1"] > row['sma_14'] and row["low_x"] <= row["S2"] :
        return "buy"
    elif row['prev_sma_11'] < row['sma_114'] and row["prev_sma_1"] < row['sma_14'] and row["high_x"] >= row["R2"]:
        return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
merged_df1 = pd.merge_asof(ohlc_df, hourly_df, on='time', direction='backward')
merged_df = pd.merge_asof(merged_df1, daily_df, on='time', direction='backward')
merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, hourly_df = hourly_df)

# Tag first signal per day in original merged_df
merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
merged_df[['signal', 'time']].dropna()

,signal,time
33766,sell,2025-05-19 10:27:00
42398,buy,2025-05-27 10:27:00
54211,buy,2025-06-06 15:30:00


In [16]:
merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "low_y": "hourly_low",
    "high_y": "hourly_high",
    "open_y": "hourly_open",
    "close_y": "hourly_close",
    "low": "daily_low",
    "high": "daily_high",
    "open": "daily_open",
    "close": "daily_close"
})


In [17]:
print(merged_df.columns.tolist())
merged_df


['time', 'open', 'high', 'low', 'close', 'hourly_open', 'hourly_high', 'hourly_low', 'hourly_close', 'ohlc4_x', 'sma_1', 'prev_sma_1', 'sma_14', 'daily_open', 'daily_high', 'daily_low', 'daily_close', 'pivot', 'prange', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3', 'ohlc4_y', 'sma_11', 'prev_sma_11', 'sma_114', 'sell_stope_loss', 'buy_stope_loss', 'sell_profit', 'buy_profit', 'signal', 'is_first_signal']


,time,open,high,low,close,hourly_open,hourly_high,hourly_low,hourly_close,ohlc4_x,...,ohlc4_y,sma_11,prev_sma_11,sma_114,sell_stope_loss,buy_stope_loss,sell_profit,buy_profit,signal,is_first_signal
0,2025-04-15 23:00:00,1.12823,1.12848,1.12823,1.12848,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
1,2025-04-15 23:01:00,1.12847,1.12847,1.12824,1.12839,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2,2025-04-15 23:02:00,1.12838,1.12862,1.12837,1.12862,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,2025-04-15 23:03:00,1.12862,1.12862,1.12837,1.12848,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
4,2025-04-15 23:04:00,1.12848,1.12848,1.12836,1.12839,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62477,2025-06-16 09:26:00,1.15486,1.15496,1.15474,1.15474,1.1536,1.15865,1.15358,1.1578,1.155907,...,1.155265,1.155265,1.155892,1.142621,1.16921,1.141103,False,False,None,NaN
62478,2025-06-16 09:27:00,1.15474,1.15474,1.15416,1.15427,1.1536,1.15865,1.15358,1.1578,1.155907,...,1.155265,1.155265,1.155892,1.142621,1.16921,1.141103,False,False,None,NaN
62479,2025-06-16 09:28:00,1.15427,1.15439,1.15405,1.15417,1.1536,1.15865,1.15358,1.1578,1.155907,...,1.155265,1.155265,1.155892,1.142621,1.16921,1.141103,False,False,None,NaN
62480,2025-06-16 09:29:00,1.15417,1.15429,1.15405,1.15429,1.1536,1.15865,1.15358,1.1578,1.155907,...,1.155265,1.155265,1.155892,1.142621,1.16921,1.141103,False,False,None,NaN


In [18]:
# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    if 'signal' not in data:
        print("Warning: 'signal' column is missing!")
        return
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        stop_loss_pips = data['S2'] - (data["S2"] - ((data["R1"] - data["S2"]) /2)) # example SL
        volume = risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        stop_loss_pips = ((data["R2"] - data["S1"]) /2 + data["R2"]) - data["R2"]
        volume = risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and ((data["low"] <= data["S2"] - ((data["R1"] - data["S2"]) /2))):
                                            #or (data["high"] >= data["R1"])):
                                            #or (data['time'].time() == pd.to_datetime('00:00:00').time())):
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and ((data["high"] >= (data["R2"] - data["S1"]) /2 + data["R2"])):
                                            #or (data["low"] <= data["S1"])):
                                            #or (data['time'].time() == pd.to_datetime('00:00:00').time())):
            orders.close_trade(trade)

In [19]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(merged_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,sell,45197.740113,2025-05-19 10:27:00,1.12241,2025-05-19 12:07:00,1.12689,0,0,{},-202.49,-3.163842,-205.653842,-205.653842,9794.346158
1,closed,EURUSD,buy,68143.100511,2025-05-27 10:27:00,1.13536,2025-05-29 02:37:00,1.1247,0,0,{},-726.41,-4.770017,-731.180017,-936.833859,9063.166141
2,closed,EURUSD,buy,44198.895028,2025-06-06 15:30:00,1.1407,2025-06-16 09:30:00,1.15441,0,0,{},605.97,-3.093923,602.876077,-333.957781,9666.042219


In [20]:
mm = bt.plot_balance()
display(mm)

In [21]:
pnl_chart = bt.plot_pnl()
display(pnl_chart)

In [22]:
evaluate_backtest(bt.trades)

biggest_profit: 605.97
daily_drawdown: -726.41
max_drawown: -731.18
avg_win: 605.97
avg_loss: -464.45
count_profit_trades: 1
count_loss_trades: 2
Win Rate1: 33.33%
win_rate 0.5
rrr: 1.3


,order_type,profit
0,buy,-120.44
1,sell,-202.49


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,current_max,drawdown,dayofweek,month,year
0,closed,EURUSD,sell,45197.740113,2025-05-19 10:27:00,1.12241,2025-05-19 12:07:00,1.12689,0,0,...,-202.49,-3.163842,-205.653842,-205.653842,9794.346158,-205.653842,0.0,0,2025-05,2025
1,closed,EURUSD,buy,68143.100511,2025-05-27 10:27:00,1.13536,2025-05-29 02:37:00,1.1247,0,0,...,-726.41,-4.770017,-731.180017,-936.833859,9063.166141,-205.653842,-731.180017,3,2025-05,2025
2,closed,EURUSD,buy,44198.895028,2025-06-06 15:30:00,1.1407,2025-06-16 09:30:00,1.15441,0,0,...,605.97,-3.093923,602.876077,-333.957781,9666.042219,-205.653842,-128.30394,0,2025-06,2025


ValueError: Plotly Express cannot process wide-form data with columns of different type.

In [ ]:
# visualizing the ohlc data
fig = go.Figure(data = [go.Candlestick(x = ohlc_df['time'],
                                        open = ohlc_df['open'],
                                        high = ohlc_df['high'],
                                        low = ohlc_df['low'],
                                        close = ohlc_df['close'])])

# fig.update_layout(height=600, xaxis_rangeslider_visible=False)
# 📌 Add Pivot Points as Horizontal Lines
pivot_levels = ["sell_stope_loss", "R2", "buy_stope_loss", "S2"]
colors = {'sell_stope_loss': 'red', 'buy_stope_loss': 'red', 'R2': 'green', 'S2': 'green'}


for _, row in daily_df.iterrows():
    for level in pivot_levels:
        fig.add_trace(go.Scatter(
            x=[row['time'], row['time'] + pd.Timedelta(days=1)],  # Extend for one day
            y=[row[level], row[level]],
            mode='lines',
            name=f"{level} {row['time']}",
            line=dict(color=colors[level], width=1.5, dash='dot')
        ))

# 📌 Customize Layout
fig.update_layout(
    title="Pivot Points on Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis_rangeslider_visible=False
)
fig